In [1]:
import json
from sklearn.externals import joblib
import os
import re
import numpy as np

In [2]:
dataPath = '../data'
trainTfIdfPath = '../data/trainDataTfIdf'

tfIdfNumberPagesPath = '../data/tfIdfNumberPages'
tfIdfPagesPath = '../data/tfidfPages'
tfIdfFeaturesPath = '../data/tfidfFeatutes'
docUrlsPath = '../data/urls.docs.txt'
textdataPath = '../data/textdata'
templateJson = '{:d}.json'

queriesDocsPath = '../data/queries.docs.txt'
queriesPath = '../data/queries.numerate.txt'

In [3]:
docsUrlsPath = '../data/urls.docs.txt'
textdataPath = '../data/textdata'
templateJson = '{:d}.json'
queriesDocsPath = '../data/queries.docs.txt'
queriesPath = '../data/queries.numerate.txt'

pat = re.compile(r'\d+')
procNumbs = sorted(list(map(lambda x: int(pat.search(x).group(0)), os.listdir(textdataPath))))

queriesDict = {}
with open(queriesPath) as inputFile:
    for line in inputFile:
        number, query = line.strip().split('\t')
        queriesDict[number] = query
        
queriesDocsDict = json.load(open(queriesDocsPath))

docsUrlsDict = {}
with open(docsUrlsPath) as inputFile:
    for line in inputFile:
        number, url, path = line.strip().split('\t')
        docsUrlsDict[number] = (url, path)

# TFIDF

Создание обучающего множества для TfidfVectorizer

In [69]:
data = {"numbs":[], "strings":[]}
sizeMemory = 0
for numb in procNumbs:
    filename = templateJson.format(numb)
    path = '{:s}/{:s}'.format(textdataPath, filename)
    pageDict = json.load(open(path))
    text = pageDict['text'].get('p', [])
    text = '\n'.join(text)
    
    data["numbs"].append(numb)
    data["strings"].append(text)
    sizeMemory += len(text)
    if sizeMemory > 10 ** 9:
        break    

In [73]:
json.dump(data, open(trainTfIdfPath, 'w'))

In [74]:
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.tokenize import TweetTokenizer

rusStopwords = stopwords.words('russian')
tokenizer = TweetTokenizer().tokenize
tfIdfVect = TfidfVectorizer(tokenizer=tokenizer, stop_words=rusStopwords, max_features=50000000)
tfIdfVect.fit(data["strings"])

TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=50000000, min_df=1,
        ngram_range=(1, 1), norm='l2', preprocessor=None, smooth_idf=True,
        stop_words=['и', 'в', 'во', 'не', 'что', 'он', 'на', 'я', 'с', 'со', 'как', 'а', 'то', 'все', 'она', 'так', 'его', 'но', 'да', 'ты', 'к', 'у', 'же', 'вы', 'за', 'бы', 'по', 'только', 'ее', 'мне', 'было', 'вот', 'от', 'меня', 'еще', 'нет', 'о', 'из', 'ему', 'теперь', 'когда', 'даже', 'ну', 'вдруг', '...гда', 'лучше', 'чуть', 'том', 'нельзя', 'такой', 'им', 'более', 'всегда', 'конечно', 'всю', 'между'],
        strip_accents=None, sublinear_tf=False,
        token_pattern='(?u)\\b\\w\\w+\\b',
        tokenizer=<bound method TweetTokenizer.tokenize of <nltk.tokenize.casual.TweetTokenizer object at 0x7f896ce3bac8>>,
        use_idf=True, vocabulary=None)

** Сохранение модели и результатов **

In [75]:
joblib.dump(tfIdfVect, tfIdfVectorizerPath)

['../data/models/tfIdfVectorizer.pkl']

In [76]:
result = tfIdfVect.transform(data["strings"])
json.dump(data["numbs"], open(tfIdfNumberRagesPath, 'w'))
json.dump(tfIdfVect.get_feature_names(), open(tfIdfFeaturesPath, 'w'))
joblib.dump(result, tfIdfRagesPath)

In [83]:
json.dump(tfIdfVect.get_feature_names(), open(tfIdfFeaturesPath, 'w'))

** Вычисление tfidf по запросам **

In [6]:
pat = re.compile(r'\d+')
procNumbs = set(map(lambda x: int(pat.search(x).group(0)), os.listdir(textdataPath)))

queriesDict = {}
with open(queriesPath) as inputFile:
    for line in inputFile:
        number, query = line.strip().split('\t')
        queriesDict[number] = query
        
queriesDocsDict = json.load(open(queriesDocsPath))

In [ ]:
class TfIdfBank:
    tfIdfNumberPagesPath = '../data/tfIdfNumberPages'
    tfIdfFeaturesPath = '../data/tfidfFeatutes'
    
    def __init__(self, tfIdfMatrix):
        numberPages = json.load(open(TfIdfBank.tfIdfNumberPagesPath))
        features = json.load(open(TfIdfBank.tfIdfFeaturesPath))
        
        self._indexPages = np.full(27000, -1, dtype=np.int)
        self._indexPages[numberPages] = range(len(numberPages))
        self._tfIdfMatrix = tfIdfMatrix#joblib.load(TfIdfFeatures.tfIdfPagesPath)        
        self._indexFeatures = dict(map(lambda x: (x[1], x[0]), enumerate(features)))
        
    def tfidf(self, word, numberPage):
        indexPage = self._indexPages[numberPage]
        if indexPage == -1:
            return None
        indexFeature = self._indexFeatures.get(word.strip().lower(), -1)
        if indexFeature == -1:
            return 0
        return self._tfIdfMatrix[indexPage, indexFeature]    

In [10]:
tfIdfForPagePath = '../data/features/tfidfFullText'
tfidfFeatures = TfIdfFeatures()
tfidfPages = {}
for numb, query in queriesDict.items():
    numbsDocs = queriesDocsDict[numb]
    words = query.strip().split()
    tfidfs = []
    for numbDoc in numbsDocs:
        if tfidfFeatures._indexPages[numbDoc] == -1:
            tfidfs.append(None)
        else:
            tfidfs.append(sum(list(map(lambda x: tfidfFeatures.tfidf(x, numbDoc), words))))
    tfidfPages[numb] = tfidfs
json.dump(tfidfPages, open(tfIdfForPagePath, 'w'))

** Сохранение tfidf матриц от всех тегов **

In [5]:
tfIdfNumberPagesPath = '../data/tfIdfNumberPages'
tfIdfFullPagesPath = '../data/tfidfFullPages'
tfIdfVectorizerPath = '../data/models/tfIdfVectorizer.pkl'

tfidfVectorizer = joblib.load(tfIdfVectorizerPath)
tfIdfNumbers = json.load(open(tfIdfNumberPagesPath))

In [ ]:
from scipy.sparse import coo_matrix, vstack
tfidfMatrix = None
data = []
maxCount = 1000
curCount = 0
for numb in tfIdfNumbers:
    filename = templateJson.format(numb)
    path = '{:s}/{:s}'.format(textdataPath, filename)
    pageDict = json.load(open(path))
    text = []
    for value in pageDict['text'].values():
        text.extend(value)
    text = '\n'.join(text)    
    data.append(text)
    curCount += 1
    if curCount > maxCount:
        print(numb,)
        if tfidfMatrix is None:
            tfidfMatrix = tfidfVectorizer.transform(data)
        else:
            tfidfMatrix = vstack([tfidfMatrix, tfidfVectorizer.transform(data)])
        data = []
        curCount = 0

if curCount > 0:
    tfidfMatrix = vstack([tfidfMatrix, tfidfVectorizer.transform(data)])
joblib.dump(tfidfMatrix, tfIdfFullPagesPath)

** Вычисление tfidf по title, keywords, description **

In [ ]:
textdataPath = '../data/textdata'
titles = []
keywords = []
descriptions = []
for numb in procNumbs:
    filename = templateJson.format(numb)
    path = '{:s}/{:s}'.format(textdataPath, filename)
    pageDict = json.load(open(path))
    titles.append(pageDict["title"])
    keywords.append(pageDict["keywords"])
    descriptions.append(pageDict["description"])

In [ ]:
json.dump(titles, open('../data/subdata/titles', 'w'))
json.dump(keywords, open('../data/subdata/keywords', 'w'))
json.dump(descriptions, open('../data/subdata/description', 'w'))

In [ ]:
titles = json.load(open('../data/subdata/titles'))
keywords = json.load(open('../data/subdata/keywords'))
descriptions = json.load(open('../data/subdata/description'))

In [ ]:
tfIdfVectorizerPath = '../data/models/tfIdfVectorizer.pkl'
tfIdfVect = joblib.load(tfIdfVectorizerPath)

In [ ]:
tfidfDicts = [{}, {}, {}]
for i, data in enumerate([titles, keywords, descriptions]):
    bank = TfIdfBank(tfIdfVect.transform(data))
    for numb, query in queriesDict.items():
        numbsDocs = queriesDocsDict[numb]
        words = query.strip().split()
        tfidfs = []
        for numbDoc in numbsDocs:
            if bank._indexPages[numbDoc] == -1:
                tfidfs.append(None)
            else:
                tfidfs.append(sum(list(map(lambda x: bank.tfidf(x, numbDoc), words))))
        tfidfDicts[i][numb] = tfidfs        

In [ ]:
titlePath = '../data/features/tfidfTitle'
keywordsPath = '../data/features/tfidfKeywords'
descPath = '../data/features/tfidfDesc'

json.dump(tfidfDicts[0], open(titlePath, 'w'))
json.dump(tfidfDicts[1], open(keywordsPath, 'w'))
json.dump(tfidfDicts[2], open(descPath, 'w'))

** Стеминг данных **

In [4]:
from nltk.stem import SnowballStemmer
from nltk.tokenize import TweetTokenizer
normalizedataPath = '../data/normalizedata'

class TextNormalizer:
    def __init__(self, stemmer, tokenizer):
        self._stemmer = stemmer
        self._tokenizer = tokenizer
    def _stemTokens(self, tokens):
        stemmed = []
        for item in tokens:
            stemmed.append(self._stemmer.stem(item))
        return stemmed
    def _tokenize(self, text):
        tokens = self._tokenizer.tokenize(text)
        stems = self._stemTokens(tokens)
        return stems
    def normalize(self, text):
        return ' '.join(self._tokenize(text))


In [ ]:
textNormalizer = TextNormalizer(SnowballStemmer('russian'), TweetTokenizer())

for numb in procNumbs:
    if numb < 7090:
        continue
    filename = templateJson.format(numb)
    path = '{:s}/{:s}'.format(textdataPath, filename)
    pageDict = json.load(open(path))
    
    pageDict["title"] = textNormalizer.normalize(pageDict["title"])
    pageDict["description"] = textNormalizer.normalize(pageDict["description"])
    pageDict["keywords"] = textNormalizer.normalize(pageDict["keywords"])
    for key, texts in pageDict["text"].items():
        pageDict["text"][key] = [textNormalizer.normalize(text) for text in texts]
    for key, texts in pageDict["attr"].items():
        pageDict["attr"][key] = [textNormalizer.normalize(text) for text in texts]
    
    outpath = '{:s}/{:s}'.format(normalizedataPath, filename)
    json.dump(pageDict, open(outpath, 'w'))
  

** Обучение Tfidf **

Создание обучающего множества

In [7]:
import numpy as np
tags = ["p", "div", "br", "span", "img", "h1", "h2", "h3", "h4", "h5", "h6", "b", "strong", "i"]
dataPath = '../data/textdata'
data = []
sizeMemory = 0

for numb in list(np.random.choice(np.array(procNumbs), size=len(procNumbs), replace=False)):
    filename = templateJson.format(numb)
    path = '{:s}/{:s}'.format(dataPath, filename)
    pageDict = json.load(open(path))
    text = []
    for tag in filter(lambda x: x in tags, pageDict['text'].keys()):
        text.extend(pageDict['text'][tag])
    text.extend([pageDict['title'], pageDict['description'], pageDict['keywords']])
    text = '\n'.join(text)     
    data.append(text)
    sizeMemory += len(text)
    if sizeMemory > 5 * (10 ** 8):
        break    

In [8]:
dataPath = '../data/trainNorNorm'
json.dump(data, open(dataPath, 'w'))

In [8]:
dataPath = '../data/trainNorNorm'
data = json.load(open(dataPath))

Обучение vectorizer

In [6]:
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.tokenize import TweetTokenizer
from nltk.stem import SnowballStemmer
from nltk.corpus import stopwords

stemmer  = SnowballStemmer('russian')
rusStopwords = [stemmer.stem(word) for word in stopwords.words('russian')]

tokenizer = TweetTokenizer().tokenize
tfIdfVect = TfidfVectorizer(tokenizer=tokenizer, stop_words=rusStopwords, max_features=1000000)
tfIdfVect.fit(data)


KeyboardInterrupt



In [9]:
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.tokenize import TweetTokenizer
from nltk.corpus import stopwords

rusStopwords = stopwords.words('russian')
tokenizer = TweetTokenizer().tokenize
tfIdfVect = TfidfVectorizer(tokenizer=tokenizer, stop_words=rusStopwords, max_features=1000000)
tfIdfVect.fit(data)
tfIdfVectorizerPath = '../data/models/randomNotNorm2.pkl' 
joblib.dump(tfIdfVect, tfIdfVectorizerPath)

['../data/models/randomNotNorm2.pkl']

Извлечение фич

In [11]:
from collections import defaultdict
class QueryHandler:
    def __init__(self, tags):
        self._textNormalizer = TextNormalizer(SnowballStemmer('russian'), TweetTokenizer())
        self._tags = tags
        self._docs = []
        self._texts = []
        self._queries = []
    def clean(self):
        self._docs = []
        self._texts = []
        self._queries = []
    def addDoc(self, query, doc):
        curTags = list(filter(lambda x: x in self._tags, doc['text'].keys()))
        curNames = ["text", "title", "keywords", 'description']
        text = []
        for tag in curTags:
            text.append('\n'.join(doc['text'][tag]))
        self._texts.append('\n'.join(text))
        self._texts.extend([doc['title'], 
                            doc['keywords'], 
                            doc['description']]) 
        self._queries.extend([queriesDict[query]] * 4)
        self._docs.append({"names":curNames, "number":doc["number"], "query":query})
    def queriesAndTexts(self):
         return (self._queries, self._texts)
    def accordForTags(self, scores):
        scoreDict = defaultdict(dict)
        curI = 0
        for doc in self._docs:
            curDict = {}
            for tag in doc["names"]:
                curDict[tag] = scores[curI]
                curI += 1
            scoreDict[doc["query"]][doc["number"]] = curDict
        return scoreDict

In [12]:
from scipy.sparse import csr_matrix
class TfidfHandler:
    def __init__(self, vectorizer):        
        self._vectorizer = vectorizer
        features = tfIdfVect.get_feature_names()
        self._indexFeatures = dict(map(lambda x: (x[1], x[0]), enumerate(features)))
    def scores(self, queries, texts):
        score = []
        matrix = self._vectorizer.transform(texts)
        for i, query in enumerate(queries):        
            wordsQuery = query.strip().split()
            curScore = 0
            for word in wordsQuery:
                j = self._indexFeatures.get(word, None)
                if j is None:
                    continue 
                curScore += matrix[i, j]
            score.append(curScore)
        return score

In [ ]:
tfIdfVectorizerPath = '../data/models/randomNotNorm.pkl' 
tfIdfVect = joblib.load(tfIdfVectorizerPath)

In [10]:
textNormalizer = TextNormalizer(SnowballStemmer('russian'), TweetTokenizer())
for numbQuery, query in queriesDict.items():
    queriesDict[numbQuery] = textNormalizer.normalize(query)    

In [ ]:
from collections import defaultdict
tags = ["p", "div", "br", "span", "img", "h1", "h2", "h3", "h4", "h5", "h6", "b", "strong", "i"]
dataPath = '../data/textdata'

tfidfsForQuery = defaultdict(dict)
queryHandler = QueryHandler(tags)
tfidfHandler = TfidfHandler(tfIdfVect)
maxCount = 1000
curCount = 0

for numbQuery, query in queriesDict.items():
    numbsDocs = queriesDocsDict[numbQuery]
    for numbDoc in numbsDocs:
        if not(numbDoc in procNumbs):
            tfidfsForQuery[numbQuery][numbDoc] = None
            continue
        filename = templateJson.format(numbDoc)
        path = '{:s}/{:s}'.format(dataPath, filename)
        doc = json.load(open(path))
        queryHandler.addDoc(numbQuery, doc)
        curCount +=1
        if curCount >= maxCount:
            print('Ok')
            xqueries, xtexts = queryHandler.queriesAndTexts()
            xscores = tfidfHandler.scores(xqueries, xtexts)
            xscores = queryHandler.accordForTags(xscores)
            for nq, xdocs in xscores.items():
                for nd, s in xdocs.items():
                    tfidfsForQuery[nq][nd] = s            
            curCount = 0
            queryHandler.clean()
            
if curCount > 0:
    xqueries, xtexts = queryHandler.queriesAndTexts()
    xscores = tfidfHandler.scores(xqueries, xtexts)
    xscores = queryHandler.accordForTags(xscores)
    for nq, xdocs in xscores.items():
        for nd, s in xdocs.items():
            tfidfsForQuery[nq][nd] = s
thidfsPath = '../data/features/tfidfsNotNorm'
json.dump(tfidfsForQuery, open(tfidfsPath, 'w'))